In [21]:
from nytimesarticle import articleAPI
import ner
import os
import re
import csv
from urllib import urlopen
import sys
csv.field_size_limit(sys.maxsize)

131072

## Prep

In [4]:
api = articleAPI('e15cea455f73cc47d6d971667e09c31c:19:44644296')

In [44]:
def get_articles(country, year):
    ''' Takes in a country and year (both strings) and returns the number of articles 
    in the New York Times about that country / year'''
    articles = api.search( fq = {'source':['The New York Times'], 
                             'glocations.contains':country}, 
                            begin_date = year + '0101', 
                            end_date= year + '1231', 
                            facet_field = ['source'], facet_filter = True )
    source_count = []
    for item in articles['response']['facets']['source']['terms']:
        source = {}
        count = int(item['count'])
        pub = item['term']
        source[pub] = count
        source_count.append(source)
    return(source_count)

## Load country-year data

In [26]:
#read csv and read into a list of dictionaries
women = []
with open('../Data/country-year-obs.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        women.append(row)
women[1:10]

[{'ccode': '540', 'country': 'Angola', 'iso3c': 'AGO', 'year': '1979'},
 {'ccode': '339', 'country': 'Albania', 'iso3c': 'ALB', 'year': '1979'},
 {'ccode': '696',
  'country': 'United Arab Emirates',
  'iso3c': 'ARE',
  'year': '1979'},
 {'ccode': '160', 'country': 'Argentina', 'iso3c': 'ARG', 'year': '1979'},
 {'ccode': '900', 'country': 'Australia', 'iso3c': 'AUS', 'year': '1979'},
 {'ccode': '305', 'country': 'Austria', 'iso3c': 'AUT', 'year': '1979'},
 {'ccode': '516', 'country': 'Burundi', 'iso3c': 'BDI', 'year': '1979'},
 {'ccode': '211', 'country': 'Belgium', 'iso3c': 'BEL', 'year': '1979'},
 {'ccode': '434', 'country': 'Benin', 'iso3c': 'BEN', 'year': '1979'}]

## Scrape NYT

In [68]:
for i in women[1000:]:
    country = i['country']
    year = i['year']
    print 'processing ' + country + '-' + year + '...'
    try:
        i['count'] = get_articles(country, year)
    except:
        i['count'] = 'NA'

processing Denmark-1985...
processing Dominican Republic-1985...
processing Algeria-1985...
processing Ecuador-1985...
processing Egypt-1985...
processing Eritrea-1985...
processing Spain-1985...
processing Ethiopia-1985...
processing Finland-1985...
processing Fiji-1985...
processing France-1985...
processing Gabon-1985...
processing United Kingdom-1985...
processing Ghana-1985...
processing Guinea-1985...
processing Gambia-1985...
processing Guinea-Bissau-1985...
processing Equatorial Guinea-1985...
processing Greece-1985...
processing Grenada-1985...
processing Guatemala-1985...
processing Guyana-1985...
processing Honduras-1985...
processing Haiti-1985...
processing Hungary-1985...
processing Indonesia-1985...
processing India-1985...
processing Ireland-1985...
processing Iran-1985...
processing Iraq-1985...
processing Iceland-1985...
processing Israel-1985...
processing Italy-1985...
processing Jamaica-1985...
processing Jordan-1985...
processing Japan-1985...
processing Kenya-198

In [67]:
women[999]

{'ccode': '54',
 'count': [{u'The New York Times': 4}],
 'country': 'Dominica',
 'iso3c': 'DMA',
 'year': '1985'}

## Deal with missing obs.

In [82]:
# change empty list to NA
for i in women:
    if not i['count']:
        i['count'] = 'NA'

In [130]:
# make a list of bad countries
bad_countries = []
for i in women:
    if i['count'] == 'NA':
        bad_countries.append(i['country'])
bad_countries = set(bad_countries)

In [131]:
# search for articles with country over 20 year period. If facet == 0, it's probably a bad country:
def get_country(country):
    ''' Takes in a country (strings) and returns the number of articles 
    in the New York Times about that country from 1980-2014'''
    articles = api.search( fq = {'source':['The New York Times'], 
                             'glocations.contains':country}, 
                            begin_date = '19800101', 
                            end_date= '20141231', 
                            facet_field = ['source'], facet_filter = True )
    source_count = []
    for item in articles['response']['facets']['source']['terms']:
        source = {}
        count = int(item['count'])
        pub = item['term']
        source[pub] = count
        source_count.append(source)
    return(source_count)

In [132]:
bad = {}
for i in bad_countries:
    try:
        bad[i] = get_country(i)
    except:
        continue

In [140]:
bad

{'Afghanistan': [{u'The New York Times': 13356}],
 'Andorra': [{u'The New York Times': 22}],
 'Bahamas': [{u'The New York Times': 8}],
 'Bahrain': [{u'The New York Times': 628}],
 'Barbados': [{u'The New York Times': 138}],
 'Belarus': [{u'The New York Times': 443}],
 'Benin': [{u'The New York Times': 73}],
 'Bhutan': [{u'The New York Times': 126}],
 'Bosnia': [{u'The New York Times': 5036}],
 'Botswana': [{u'The New York Times': 178}],
 'Brunei': [{u'The New York Times': 81}],
 'Burkina Faso': [{u'The New York Times': 116}],
 'Burundi': [{u'The New York Times': 249}],
 'Cameroon': [{u'The New York Times': 195}],
 'Cape Verde': [{u'The New York Times': 29}],
 'Central African Republic': [{u'The New York Times': 237}],
 'Chad': [{u'The New York Times': 552}],
 'Comoros': [{u'The New York Times': 21}],
 'Congo-Brazzaville': [{u'The New York Times': 2}],
 'Congo-Kinshasa': [{u'The New York Times': 2}],
 'Djibouti': [{u'The New York Times': 50}],
 'Dominica': [{u'The New York Times': 56}],

In [122]:
# function to replace country names
def replace_country(bad, good):
    for i in women:
        if i['country'] == bad:
            i['country'] = good

In [129]:
replace_country("Bosnia and Herzegovina", "Bosnia")
replace_country("Democratic Republic of the Congo","Congo-Kinshasa")
replace_country("Congo","Congo-Brazzaville")
replace_country("Federated States of Micronesia", "Micronesia")
replace_country("United Kingdom", "Great Britain")

In [127]:
for i in women:
    if i['country'] == "Congo-Kinshasa":
        if int(i['year']) < 1998:
            i['country'] == "Zaire"

In [114]:
articles = api.search( fq = {'source':['The New York Times'], 
                             'glocations.contains':"Zaire"}, 
                            begin_date = '19800101', 
                            end_date= '20141231', 
                            facet_field = ['source'], facet_filter = True )

In [115]:
articles

{u'copyright': u'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 u'response': {u'docs': [{u'_id': u'4fd21d8f8eb7c8105d79eb57',
    u'abstract': u"Democracy is slowly, imperfectly, inching across Africa, continent better known for wars, disease, corruption and repression; since 1990, 42 of 48 countries in sub-Saharan Africa have held multiparty elections; in early 1980's, only four countries held multiparty elections on continent dominated by military juntas and autocrats; one dictator, Mobutu Sese Seko of Zaire, famously declared that 'democracy is not for Africa'; today, despite common complaints of vote rigging and political intimidation, most Africans have right to choose their leaders at ballot box; over last two years, governing parties in Senegal, Mauritius, Ghana and Mali have peacefully handed power to opposition party rivals; but for many ordinary Africans, tangible benefits of political change remain elusive; so far, democracy has often been followed b

## Transform to just count

In [85]:
# get rid of list, just make it the count
for i in women[:50]:
    if i['count'] != 'NA':
        i['count'] = i['count'][0].values()[0]

152
25
6
1
82
37
27
1
20
1
22
5
7
64
72
1
4
27
358
102
750
3
3
3
23
1
3
49
253
37
13
18
27
22
105
176
28
12
1
214


In [69]:
keys = women[0].keys()
with open('../Data/n_nyt.csv.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(women)